In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

2021-08-18 07:33:23.786745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-18 07:33:23.786813: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
tab_columns=['marketplace','customer_id','review_id','product_id','product_parent',
        'product_title','product_category','star_rating','helpful_votes','total_votes',
        'vine','verified_purchase','review_headline','review_body','review_date']

review_df= pd.read_csv('amazon_reviews_us_Software_v1_01.tsv',names=tab_columns, 
        dtype={'marketplace':str,'customer_id':str,'review_id':str,'product_id':str,
             'product_parent':str, 'product_title':str, 'product_category':str,
             'star_rating':str,'helpful_votes':str,'total_votes':str, 'vine':str,
             'verified_purchase':str,'review_headline':str,'review_body':str,'review_date':str},
                       sep='\t')
columns=review_df.iloc[-1]
#review_df.drop(columns=['marketplace','product_category'], inplace=True)
#review_df.drop(index='product_id',inplace=True)
review_df.dropna(inplace=True,subset=['review_body'])
review_df.set_index('product_id',inplace=True)
review_df.sort_index(inplace=True)


KeyError: -1

In [17]:
columns=review_df.iloc[-1]
columns

marketplace                marketplace
customer_id                customer_id
review_id                    review_id
product_parent          product_parent
product_title            product_title
product_category      product_category
star_rating                star_rating
helpful_votes            helpful_votes
total_votes                total_votes
vine                              vine
verified_purchase    verified_purchase
review_headline        review_headline
review_body                review_body
review_date                review_date
Name: product_id, dtype: object

In [12]:
review_df

,marketplace,customer_id,review_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
product_id,,,,,,,,,,,,,,
0028650506,US,39107575,RING4VEIUW4Y1,708318734,BLACK MUSIC OF TWO WORLDS CD,Software,1,0,0,N,N,teachers and students BEWARE !!!,I have enjoyed and admired John Storm Robert's...,2004-10-11
0030331927,US,33712978,R38LNO9CFVPKQS,524450836,INTERMEDIATE FIN MGMT 7E,Software,5,3,15,N,N,one of the best sources of corporate finance,one of the best sources of corporate finance,2004-03-01
0071480935,US,12696548,RB2EAMTUSLP1V,64378012,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",Software,5,2,2,N,Y,Grown ups even like it!,"My husband and I are in our 50's, are beginnin...",2011-08-06
0071480935,US,33655878,R2IE0O9FYKMKL,64378012,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",Software,5,3,3,N,N,Great For Toddlers,I bought this video for my 20-month old grandd...,2008-07-12
0071480935,US,4138699,R131XDRSFWXBGA,64378012,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",Software,5,0,0,N,Y,This is one of my favorite kids spanish learni...,This is great immersion type learning for the ...,2014-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B013ETQE9Y,US,107366,R18VRLNW9D9UAM,578141346,Microsoft Office Home & Student 2013 32/64-bit...,Software,5,0,1,N,N,Good product as always from microsoft,Good product as always from microsoft. A littl...,2015-08-27
B013KEZ598,US,33974036,R37543ALZA2XRA,28034061,Windows Xp Professional 32-bit New Reinstall O...,Software,1,0,0,N,N,Fake Pirated Microsoft Software,Fake pirated software. Microsoft legit softwar...,2015-08-23
B014E4FBCO,US,31473739,RR8BFVK4UA96S,9681631,Windows 7 Professional Sp1 32bit DVD 1 Pack wi...,Software,5,2,2,N,N,Five Stars,Excellent worked and activated,2015-08-26


In [2]:
import pickle


In [ ]:
review_df.columns

In [12]:
stop_words=spacy.lang.en.stop_words.STOP_WORDS
def spacy_tok(text):
#for i in range(1):
    ''' Use Spacy library to perform processing and lemmentation.
    '''
    #remove digit
    text=re.sub('\w*\d\w*', ' ', text)
    #remove punctuation
    text=re.sub('[%s]' % re.escape(string.punctuation), ' ',text)
#    parser=spacy.lang.en.English()
    tokens=nlp(text)
    tokens=[ word.lemma_.lower().strip() for word in tokens  ]
    tokens=[ word  for word in tokens if (word not in stop_words) ]
    return tokens 

In [ ]:
review_df.info()

In [19]:
example=review_df['review_body'].tolist()
#tokenizer=CountVectorizer
vectorizer=TfidfVectorizer

In [20]:
result = vectorizer(tokenizer=spacy_tok, max_df=0.7, min_df=5,ngram_range=(1,2),lowercase=False)
doc_word = result.fit_transform(example)
doc_word.shape


(341299, 27720)

In [22]:
result = vectorizer(tokenizer=spacy_tok, max_df=0.7, min_df=5,ngram_range=(1,1),lowercase=False)
doc_word = result.fit_transform(example)
doc_word.shape

(341299, 355365)

In [3]:
#pickle.dump(doc_word,open('doc_term2.p','wb'))
#pickle.dump(result,open('result2.p','wb'))
doc_word=pickle.load(open('doc_term.p','rb'))
doc_topic=pickle.load(open('topicnmf.p','rb'))

In [25]:
lsa = TruncatedSVD(50)
doc_topic = lsa.fit_transform(doc_word)
pickle.dump(doc_topic,open('topiclsa.p','wb'))
pickle.dump(lsa,open('modelsvd.p','wb'))

In [8]:
lsa = NMF(50)
doc_topic = lsa.fit_transform(doc_word)
pickle.dump(doc_topic,open('topicnmf.p','wb'))
pickle.dump(lsa,open('modelnmf.p','wb'))

/home/lin/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [ ]:
text="example[1] <br> <hj/> \5 john89 text head we his our"
tokens=spacy_tok(text)
tokens

In [13]:
result=pickle.load(open('result.p','rb'))

In [7]:
ddoc_topic.shape

(341299, 50)

In [14]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [15]:
display_topics(lsa, result.get_feature_names(), 7)


Topic  0
program, card, run, find, print, design, want

Topic  1
br, identify, review, screen, feature, course, start

Topic  2
great, deal, value, job, service, tool, protection

Topic  3
good, far, deal, job, value, pretty, condition

Topic  4
game, play, fun, enjoy, graphic, kid, puzzle

Topic  5
year, ve, ago, turbotax, past, change, taxis

Topic  6
work, fine, perfectly, advertise, doesn, didn, nt

Topic  7
security, internet, kaspersky, trend, micro, suite, pc

Topic  8
use, ve, simple, find, ease, difficult, continue

Topic  9
version, previous, early, update, late, trial, deluxe

Topic  10
product, key, satisfied, review, receive, seller, company

Topic  11
love, son, kid, daughter, gift, absolutely, wife

Topic  12
windows, xp, vista, run, microsoft, os, system

Topic  13
software, package, piece, company, design, look, come

Topic  14
t, don, didn, doesn, know, haven, win

Topic  15
easy, follow, understand, quick, taxis, set, navigate

Topic  16
need, exactly, business, mee

In [ ]:
for i in 

In [16]:
result.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': False,
 'max_df': 0.7,
 'max_features': None,
 'min_df': 5,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.spacy_tok(text)>,
 'use_idf': True,
 'vocabulary': None}

In [ ]:
print(chr(0x1a))

In [ ]:
pd.DataFrame(doc_word.toarray(), columns=result.get_feature_names()).head()

In [32]:
result2=pickle.load(open('result2.p','rb'))

In [33]:
result2.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': False,
 'max_df': 0.7,
 'max_features': None,
 'min_df': 5,
 'ngram_range': (1, 2),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.spacy_tok(text)>,
 'use_idf': True,
 'vocabulary': None}